In [13]:
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np

## Preparación de los datos

In [2]:
## No se si hace falta para el dataset final -> tengo dos archivos csv porque me he descargado por separado las img benignas y las malignas -> concateno los csv !!!!!!!!!!!!!!!!!!!!!!!!!

# Cargar los datos de los dos archivos CSV
data_1 = pd.read_csv('dataset_nuria/metadata_1.csv')
data_2 = pd.read_csv('dataset_nuria/metadata_2.csv')

# Combinar los dos DataFrames
combined_data = pd.concat([data_1, data_2], ignore_index=True)

# Guardar el DataFrame combinado en un nuevo archivo CSV
combined_data.to_csv('dataset_nuria/metadata.csv', index=False)

In [3]:
## Crear el DataFrame necesario para ImageDataGenerator
dataset_path = 'dataset_nuria/' # Cambiar tuta del archivo CSV !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

# Cargar el archivo con las etiquetas
data = pd.read_csv(dataset_path + 'metadata.csv') 

# Crear el DataFrame con dos columnas -> ruta de la imagen | etiqueta (0:benigno, 1:melanoma) 
new_data = pd.DataFrame({'x_col': dataset_path + data['isic_id'] + '.jpg',
                        'y_col': data['benign_malignant'].apply(lambda x: 'benign' if x == 'benign' else 'malign')}) # melanoma

print(new_data)

                             x_col   y_col
0   dataset_nuria/ISIC_0085644.jpg  benign
1   dataset_nuria/ISIC_0260516.jpg  benign
2   dataset_nuria/ISIC_0296251.jpg  benign
3   dataset_nuria/ISIC_0378777.jpg  benign
4   dataset_nuria/ISIC_0438180.jpg  benign
..                             ...     ...
95  dataset_nuria/ISIC_8651648.jpg  malign
96  dataset_nuria/ISIC_9098311.jpg  malign
97  dataset_nuria/ISIC_9320992.jpg  malign
98  dataset_nuria/ISIC_9677008.jpg  malign
99  dataset_nuria/ISIC_9928278.jpg  malign

[100 rows x 2 columns]


In [4]:
# Crear el generador de datos de imagenes en tiempo real -> Aplicar una normalizacion + Separacion Train Test (20% Train y 80% Test)
datagen = ImageDataGenerator(rescale=1./255., validation_split=0.20)

batch_size = 20 # Lo ponen a 128 pero con 100 imagenes no puedo ponerlo a tanto!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

# Generador de datos de entrenamiento
train_generator = datagen.flow_from_dataframe(
    dataframe=new_data,
    x_col='x_col',
    y_col='y_col',
    target_size=(224,224),
    color_mode='rgb',
    batch_size=batch_size,
    class_mode='categorical',
    subset='training',
    shuffle=True
)

# Generador de datos de validacion
test_generator = datagen.flow_from_dataframe(
    dataframe=new_data,
    x_col='x_col',
    y_col='y_col',
    target_size=(224,224),
    color_mode='rgb',
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation',
    shuffle=True
)

Found 80 validated image filenames belonging to 2 classes.
Found 20 validated image filenames belonging to 2 classes.


## Modelo VGG16

In [23]:
# Dimensiones de las imagenes
input = Input((224, 224, 3)) 

## Capas Convolucionales
# Bloque 1
conv1  = Conv2D(filters=64, kernel_size=(3,3), padding="same", activation="relu")(input)
conv2  = Conv2D(filters=64, kernel_size=(3,3), padding="same", activation="relu")(conv1)
pool1  = MaxPooling2D((2, 2))(conv2)

# Bloque 2
conv3  = Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu")(pool1)
conv4  = Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu")(conv3)
pool2  = MaxPooling2D((2, 2))(conv4)

# Bloque 3
conv5  = Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu")(pool2)
conv6  = Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu")(conv5)
conv7  = Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu")(conv6)
pool3  = MaxPooling2D((2, 2))(conv7)

# Bloque 4
conv8  = Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu")(pool3)
conv9  = Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu")(conv8)
conv10 = Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu")(conv9)
pool4  = MaxPooling2D((2, 2))(conv10)

# Bloque 5
conv11 = Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu")(pool4)
conv12 = Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu")(conv11)
conv13 = Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu")(conv12)
pool5  = MaxPooling2D((2, 2))(conv13)

## Capas Densas
flat   = Flatten()(pool5)
dense1 = Dense(4096, activation="relu")(flat)
drop1  = Dropout(0.5)(dense1)
dense2 = Dense(4096, activation="relu")(drop1)
drop2  = Dropout(0.5)(dense2)
output = Dense(1, activation="sigmoid")(drop2)  # Clasificacion binaria

# Modelo
vgg_16_model = Model(inputs=input, outputs=output)
vgg_16_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
vgg_16_model.summary()

Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv2d_65 (Conv2D)          (None, 224, 224, 64)      1792      
                                                                 
 conv2d_66 (Conv2D)          (None, 224, 224, 64)      36928     
                                                                 
 max_pooling2d_25 (MaxPooli  (None, 112, 112, 64)      0         
 ng2D)                                                           
                                                                 
 conv2d_67 (Conv2D)          (None, 112, 112, 128)     73856     
                                                                 
 conv2d_68 (Conv2D)          (None, 112, 112, 128)     147584    
                                                           

## Entrenamiento

In [24]:
# Detiene el entrenamiento si la precision de la validacion no mejora en despues de 2 epocas
early_stop = EarlyStopping(monitor="val_loss", patience=2, verbose=1)

# Guarda el modelo con mejor precision en la validacion
mcp = ModelCheckpoint('modelVVG.h5', verbose=1)

# Parametros !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
train_steps = train_generator.n // train_generator.batch_size
test_steps = test_generator.n // test_generator.batch_size
epochs = 30

# Entrenamiento del modelo
history = vgg_16_model.fit(train_generator, steps_per_epoch=train_steps, epochs=epochs, validation_data=test_generator, validation_steps=test_steps, verbose=1, callbacks=[mcp,early_stop])

Epoch 1/30
4/4 [==============================] - ETA: 0s - loss: 1.4176 - accuracy: 0.6500 
Epoch 1: saving model to modelVVG.h5


c:\Users\nuria\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


4/4 [==============================] - 190s 46s/step - loss: 1.4176 - accuracy: 0.6500 - val_loss: 1.2223 - val_accuracy: 0.0000e+00
Epoch 2/30
4/4 [==============================] - ETA: 0s - loss: 0.6711 - accuracy: 0.6250 
Epoch 2: saving model to modelVVG.h5
4/4 [==============================] - 183s 47s/step - loss: 0.6711 - accuracy: 0.6250 - val_loss: 0.8285 - val_accuracy: 0.0000e+00
Epoch 3/30
4/4 [==============================] - ETA: 0s - loss: 0.6719 - accuracy: 0.6250 
Epoch 3: saving model to modelVVG.h5
4/4 [==============================] - 168s 41s/step - loss: 0.6719 - accuracy: 0.6250 - val_loss: 0.8794 - val_accuracy: 0.0000e+00
Epoch 4/30
4/4 [==============================] - ETA: 0s - loss: 0.6580 - accuracy: 0.6250 
Epoch 4: saving model to modelVVG.h5
4/4 [==============================] - 169s 42s/step - loss: 0.6580 - accuracy: 0.6250 - val_loss: 1.1009 - val_accuracy: 0.0000e+00
Epoch 4: early stopping


## Métricas

In [6]:
## Graficos del Accuracy y la perdida
train_accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']
train_loss = history.history['loss']
val_loss = history.history['val_loss']

# Plot Accuracy
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(train_accuracy, label='Train Accuracy')
plt.plot(val_accuracy, label='Validation Accuracy')
plt.title('Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

# Plot Perdida
plt.subplot(1, 2, 2)
plt.plot(train_loss, label='Train Loss')
plt.plot(val_loss, label='Validation Loss')
plt.title('Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()

NameError: name 'history' is not defined

In [7]:
## Cargar el modelo -> No se si sera necesario !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
model = load_model('./modelVVG.h5') 

In [14]:
## Matriz de confusion

# Predicciones para el conjunto de datos de test
predictions = model.predict(test_generator, verbose=1) # cambiar por vgg_16_model.predict !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
predictions = predictions.argmax(axis=1)
print('Preditions: \n', predictions)

# Etiquetas del conjunto de datos de test (GT)
test_labels = test_generator.classes
print('Ground Truth: \n', test_labels)

# Matriz de confusión
matrix = confusion_matrix(test_labels, predictions)
print('Confusion matrix: \n', matrix)

1/1 [==============================] - 11s 11s/step
Preditions: 
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Ground Truth: 
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Confusion matrix: 
 [[20]]


In [15]:
print(classification_report(test_generator.classes, predictions, zero_division=0))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        20

    accuracy                           1.00        20
   macro avg       1.00      1.00      1.00        20
weighted avg       1.00      1.00      1.00        20

